In [29]:

from HPS_regular import RegularHeatPumpStudy
from HPS_multistage_condenser import InternalCondenserHeatPumpStudy
from HeatPumpStudy import HeatPumpStudy
from read_csv import read_energy_mix_csv, read_hdd_csv
import pandas as pd
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Assumptions:
- no water heating
- no cooling
- temperature over the whole month is constant
- heat consumption is constant (heat pump is running all the time)
- COP is constant over the whole month
- no losses due to variable speed
- minimum temperature lift of 20°C for the heatpump
- Heat pump with the ideal required capacity is available 

In [30]:
working_fluid="R290" # Propane

# Monthly energy mix in Germany in 2022
# Source: https://www.energy-charts.info/charts/energy/chart.htm?l=de&c=DE&chartColumnSorting=default&source=total&stacking=stacked_percent&partsum=1&interval=month&month=-1&year=2022
file_path = "source_data/energy_mix_2022.csv"
monthly_energy_mix = read_energy_mix_csv(file_path)

# Monthly Heating degree days in germany (in Celsius)
# Source: https://ec.europa.eu/eurostat/databrowser/view/NRG_CHDD_M__custom_6030336/default/table?lang=en
file_path = 'source_data/monthly_hdd_18-22.csv'
monthly_hdd = read_hdd_csv(file_path)

# Monthly average ambient temperatures in Germany (in Celsius)
# Source: https://www.timeanddate.com/weather/germany/berlin/climate
monthly_ambient_temps = [1, 2, 5, 9, 14, 17, 19, 18, 14, 10, 5, 2]

electricity_cost = 0.40 # prognosis until 2040 in €/kWh_el source: https://www.zfk.de/politik/deutschland/strompreis-prognose-2042-habeck-ministerium#:~:text=Strompreis%3A%2037%20bis%2042%20Cent,pro%20kWh%20im%20Jahr%202042.
electricty_cost_no_subvention =0.50 # in €/kWh_el source: https://www.bmwk.de/Redaktion/DE/Schlaglichter-der-Wirtschaftspolitik/2023/01/03-im-fokus.html#:~:text=F%C3%BCr%20private%20Haushalte%20und%20Unternehmen%20mit%20einem%20Gasverbrauch%20bis%20zu,%2C5%20Cent%20pro%20kWh).
gas_cost = 0.127 # in €/kWh_th source: https://www.bmwk.de/Redaktion/DE/Schlaglichter-der-Wirtschaftspolitik/2023/01/03-im-fokus.html#:~:text=F%C3%BCr%20private%20Haushalte%20und%20Unternehmen%20mit%20einem%20Gasverbrauch%20bis%20zu,%2C5%20Cent%20pro%20kWh).
gas_cost_no_subvention = 0.22 # in €/kWh_th source: https://www.bmwk.de/Redaktion/DE/Schlaglichter-der-Wirtschaftspolitik/2023/01/03-im-fokus.html#:~:text=F%C3%BCr%20private%20Haushalte%20und%20Unternehmen%20mit%20einem%20Gasverbrauch%20bis%20zu,%2C5%20Cent%20pro%20kWh).

# CO2 emissions per kWh for different energy sources (in kg CO2/kWh_el)
# Assumptions:
coal_co2_emissions = 0.97 #   Source: https://www.volker-quaschning.de/datserv/CO2-spez/index_e.php
natural_gas_co2_emissions = 0.43 #   Source: https://www.volker-quaschning.de/datserv/CO2-spez/index_e.php
nuclear_co2_emissions = 0.012 #   Source: https://www.ipcc.ch/site/assets/uploads/2018/02/ipcc_wg3_ar5_annex-iii.pdf (page 1335)
renewable_co2_emissions = 0.04 #   Source: https://www.ipcc.ch/site/assets/uploads/2018/02/ipcc_wg3_ar5_annex-iii.pdf (page 133)

# TODO:improvement replace assumptions about heating requirements and different insulation classes with more granular data from https://www.ise.fraunhofer.de/content/dam/ise/de/downloads/pdf/Forschungsprojekte/BMWi-03ET1272A-WPsmart_im_Bestand-Schlussbericht.pdf

# median living space area that needs heating in Germany (in m²) source: https://www.ise.fraunhofer.de/content/dam/ise/de/downloads/pdf/Forschungsprojekte/BMWi-03ET1272A-WPsmart_im_Bestand-Schlussbericht.pdf
heating_area = 183
# Heating demand for new, well-insulated homes (in kWh/m²/year) and assuming 200m² living space per heat pump
# Source: https://www.thermondo.de/info/rat/heizen/heizwaermebedarf-ermitteln/
specific_new_home_heating_demand = 30

# median Heating demand for existing homes 
# Source:  https://www.ise.fraunhofer.de/content/dam/ise/de/downloads/pdf/Forschungsprojekte/BMWi-03ET1272A-WPsmart_im_Bestand-Schlussbericht.pdf
specific_renovation_heating_demand = 110

new_home_heating_demand = specific_new_home_heating_demand * heating_area
renovation_heating_demand = specific_renovation_heating_demand * heating_area

# Heating water temperature for new homes with large radiators (in Celsius)
new_home_heating_temp =  40

# Heating water temperature for renovation with small radiators (in Celsius)
renovation_heating_temp = 65

# Base temperature for calculating HDD (in Celsius)
base_temperature = 18

dT_condenser = 5 # in Kelvin
dT_evaporator = 5 # in Kelvin

# ratio of renovation to new homes in 2022
# Source: https://www.waermepumpe.de/fileadmin/user_upload/waermepumpe/08_Sonstige/Filedump/BWP_Branchenstudie_2023_DRUCK.pdf page 11
total_sales_renovations = 175000
total_sales_new_homes = 62000
renovation_to_new_homes_ratio = total_sales_renovations / total_sales_new_homes

In [31]:
# calculate heat demand in kWh for a given number of heating degree days
def monthly_heat_demand_from_hdd(hdd, annual_heat_demand):
    return (hdd / sum(monthly_hdd)) * annual_heat_demand


def calculate_monthly_cop(study:HeatPumpStudy, heating_temp, avg_ambient_temp, monthly_heat_demand):
    # convert monthly heat demand from kWh to W assuming 100% duty cycle and 30 days per month
    study.Q_out = monthly_heat_demand / (24 * 30)*1000
    study.setup_network(iterinfo=False)
    heating_temp = max(avg_ambient_temp+20, heating_temp) # heating temp is at least 20 degrees above ambient temperature to account for minimal temperature lift
    if isinstance(study, InternalCondenserHeatPumpStudy):
        study.set_boundary_conditions(T_cond=heating_temp+dT_condenser, T_evap=avg_ambient_temp-dT_evaporator, T_consumer=heating_temp)
    else:
        study.set_boundary_conditions(T_cond=heating_temp+dT_condenser, T_evap=avg_ambient_temp-dT_evaporator)
    study.solve()
    return study.calculate_cop()

def calculate_monthly_co2_emissions(energy_mix, cop, heat_demand_kwh):
    co2_emissions_per_kwh = (
        energy_mix["coal"] * coal_co2_emissions
        + energy_mix["natural_gas"] * natural_gas_co2_emissions
        + energy_mix["nuclear"] * nuclear_co2_emissions
        + energy_mix["renewable"] * renewable_co2_emissions
    )
    return (heat_demand_kwh / cop) * co2_emissions_per_kwh

def calculate_monthly_energy_consumption(cop, heat_demand_kwh):
    return heat_demand_kwh / cop

def calculate_annual_co2_emissions(heatpump:HeatPumpStudy, annual_heating_demand, heating_temp):
    annual_co2_emissions = 0
    for i in range(12):
        monthly_heat_demand = monthly_heat_demand_from_hdd(monthly_hdd[i], annual_heating_demand)
        avg_ambient_temp = monthly_ambient_temps[i]
        cop = calculate_monthly_cop(heatpump,heating_temp, avg_ambient_temp, monthly_heat_demand)
        co2_emissions = calculate_monthly_co2_emissions(monthly_energy_mix[i], cop, monthly_heat_demand)
        #print(f"CO2 emissions for month {str(i+1)}: {str(co2_emissions)} kg")
        annual_co2_emissions += co2_emissions
    return annual_co2_emissions

def calculate_annual_energy_consumption(heatpump:HeatPumpStudy, annual_heating_demand, heating_temp):
    annual_energy_consumption = 0
    for i in range(12):
        monthly_heat_demand = monthly_heat_demand_from_hdd(monthly_hdd[i], annual_heating_demand)
        avg_ambient_temp = monthly_ambient_temps[i]
        cop = calculate_monthly_cop(heatpump,heating_temp, avg_ambient_temp, monthly_heat_demand)
        energy_consumption = calculate_monthly_energy_consumption(cop, monthly_heat_demand)
        #print(f"Energy consumption for month {str(i+1)}: {str(energy_consumption)} kWh")
        annual_energy_consumption += energy_consumption
    return annual_energy_consumption

In [32]:
def main():
    #Sales prognosis for 2026, 2027
    total_sales = [3600, 12700]
    # yearly growth of 30% beyond that
    yearly_growth = 0.3
    total_sales.extend([round(total_sales[-1] * (1 + yearly_growth) ** i) for i in range(1, 10)])
    years=len(total_sales)

    new_home_heatpumps = [round(total_sales[i] / (1 + renovation_to_new_homes_ratio)) for i in range(years)]
    renovation_heatpumps = [round(total_sales[i] /(1+1/ renovation_to_new_homes_ratio)) for i in range(years)]

    regular_heatpump=RegularHeatPumpStudy()
    regular_heatpump_new_home_annual_energy = calculate_annual_energy_consumption(regular_heatpump, new_home_heating_demand, new_home_heating_temp)
    regular_heatpump_new_home_annual_co2 = calculate_annual_co2_emissions(regular_heatpump, new_home_heating_demand, new_home_heating_temp)
    regular_heatpump_new_home_annual_energy_cost = regular_heatpump_new_home_annual_energy * electricity_cost
    regular_heatpump_renovation_annual_energy = calculate_annual_energy_consumption(regular_heatpump, renovation_heating_demand, renovation_heating_temp)
    regular_heatpump_renovation_annual_co2 = calculate_annual_co2_emissions(regular_heatpump, renovation_heating_demand, renovation_heating_temp)
    regular_heatpump_renovation_annual_energy_cost = regular_heatpump_renovation_annual_energy * electricity_cost
    print(f"regular heat pump rennovation annual energy cost",regular_heatpump_renovation_annual_energy_cost)
    
    improved_heatpump=InternalCondenserHeatPumpStudy(expansion_device="expander", N=1)
    improved_heatpump_new_home_annual_energy = calculate_annual_energy_consumption(improved_heatpump, new_home_heating_demand, new_home_heating_temp)
    improved_heatpump_new_home_annual_co2 = calculate_annual_co2_emissions(improved_heatpump, new_home_heating_demand, new_home_heating_temp)
    improved_heatpump_new_home_annual_energy_cost = improved_heatpump_new_home_annual_energy * electricity_cost
    improved_heatpump_renovation_annual_energy = calculate_annual_energy_consumption(improved_heatpump, renovation_heating_demand, renovation_heating_temp)
    improved_heatpump_renovation_annual_co2 = calculate_annual_co2_emissions(improved_heatpump, renovation_heating_demand, renovation_heating_temp)
    improved_heatpump_renovation_annual_energy_cost = improved_heatpump_renovation_annual_energy * electricity_cost

    gas_furnace_new_home_annual_energy_cost = new_home_heating_demand * gas_cost
    gas_furnace_renovation_annual_energy_cost = renovation_heating_demand * gas_cost
    

    relative_energy_savings_new_home = (1-improved_heatpump_new_home_annual_energy/regular_heatpump_new_home_annual_energy)*100
    relative_co2_savings_new_home = (1-improved_heatpump_new_home_annual_co2/regular_heatpump_new_home_annual_co2)*100
    relative_cost_savings_new_home = (1-improved_heatpump_new_home_annual_energy_cost/regular_heatpump_new_home_annual_energy_cost)*100
    relative_cost_savings_compared_to_gas_heating_new_home = (1-improved_heatpump_new_home_annual_energy_cost/gas_furnace_new_home_annual_energy_cost)*100
    relative_energy_savings_renovation = (1-improved_heatpump_renovation_annual_energy/regular_heatpump_renovation_annual_energy)*100
    relative_co2_savings_renovation = (1-improved_heatpump_renovation_annual_co2/regular_heatpump_renovation_annual_co2)*100
    relative_cost_savings_renovation = (1-improved_heatpump_renovation_annual_energy_cost/regular_heatpump_renovation_annual_energy_cost)*100
    relative_cost_savings_compared_to_gas_heating_renovation = (1-improved_heatpump_renovation_annual_energy_cost/gas_furnace_renovation_annual_energy_cost)*100
    
    print(f"heat demand in new homes: {new_home_heating_demand} kWh")
    print(f"heat demand in renovations: {renovation_heating_demand} kWh")
    print(f"cost for gas heating in new homes: {gas_furnace_new_home_annual_energy_cost} €")
    print(f"cost for gas heating in renovations: {gas_furnace_renovation_annual_energy_cost} €")
    print("")
    print(f"relative energy savings in new homes: {round(relative_energy_savings_new_home,2)}%")
    print(f"relative co2 savings in new homes: {round(relative_co2_savings_new_home,2)}%")
    print(f"relative cost savings in new homes: {round(relative_cost_savings_new_home,2)}%")
    print(f"relative cost savings compared to gas heating in new homes: {round(relative_cost_savings_compared_to_gas_heating_new_home,2)}%")
    print(f"relative energy savings in renovations: {round(relative_energy_savings_renovation,2)}%")
    print(f"relative co2 savings in renovations: {round(relative_co2_savings_renovation,2)}%")
    print(f"relative cost savings in renovations: {round(relative_cost_savings_renovation,2)}%")
    print(f"relative cost savings compared to gas heating in renovations: {round(relative_cost_savings_compared_to_gas_heating_renovation,2)}%")
    print("")

    absolute_energy_savings_new_home = regular_heatpump_new_home_annual_energy-improved_heatpump_new_home_annual_energy
    absolute_co2_savings_new_home = regular_heatpump_new_home_annual_co2-improved_heatpump_new_home_annual_co2
    absolute_cost_savings_new_home = regular_heatpump_new_home_annual_energy_cost-improved_heatpump_new_home_annual_energy_cost
    absolute_cost_savings_compared_to_gas_heating_new_home = gas_furnace_new_home_annual_energy_cost-improved_heatpump_new_home_annual_energy_cost
    absolute_energy_savings_renovation = regular_heatpump_renovation_annual_energy-improved_heatpump_renovation_annual_energy
    absolute_co2_savings_renovation = regular_heatpump_renovation_annual_co2-improved_heatpump_renovation_annual_co2
    absolute_cost_savings_renovation = regular_heatpump_renovation_annual_energy_cost-improved_heatpump_renovation_annual_energy_cost
    absolute_cost_savings_compared_to_gas_heating_renovation = gas_furnace_renovation_annual_energy_cost-improved_heatpump_renovation_annual_energy_cost
    print(f"absolute energy savings in new homes: {round(absolute_energy_savings_new_home,2)} kWh")
    print(f"absolute co2 savings in new homes: {round(absolute_co2_savings_new_home,2)} kg")
    print(f"absolute cost savings in new homes: {round(absolute_cost_savings_new_home,2)} €")
    print(f"absolute cost savings compared to gas heating in new homes: {round(absolute_cost_savings_compared_to_gas_heating_new_home,2)} €")
    print(f"absolute energy savings in renovations: {round(absolute_energy_savings_renovation,2)} kWh")
    print(f"absolute co2 savings in renovations: {round(absolute_co2_savings_renovation,2)} kg")
    print(f"absolute cost savings in renovations: {round(absolute_cost_savings_renovation,2)} €")
    print(f"absolute cost savings compared to gas heating in renovations: {round(absolute_cost_savings_compared_to_gas_heating_renovation,2)} €")
    print(f"average energy savings: {round((absolute_energy_savings_new_home*total_sales_new_homes+absolute_energy_savings_renovation*total_sales_renovations)/(total_sales_new_homes+total_sales_renovations),2)} kWh")
    print(f"average co2 savings: {round((absolute_co2_savings_new_home*total_sales_new_homes+absolute_co2_savings_renovation*total_sales_renovations)/(total_sales_new_homes+total_sales_renovations),2)} kg")
    print(f"average cost savings: {round((absolute_cost_savings_new_home*total_sales_new_homes+absolute_cost_savings_renovation*total_sales_renovations)/(total_sales_new_homes+total_sales_renovations),2)} €")


    # let's export those to a dataframe and then to an excel sheet
    single_home_results = [
        [
            "New homes",
            regular_heatpump_new_home_annual_energy,
            regular_heatpump_new_home_annual_co2,
            improved_heatpump_new_home_annual_energy,
            improved_heatpump_new_home_annual_co2,
            relative_energy_savings_new_home,
            relative_co2_savings_new_home,
            relative_cost_savings_new_home,
            relative_cost_savings_compared_to_gas_heating_new_home,
            absolute_energy_savings_new_home,
            absolute_co2_savings_new_home,
            absolute_cost_savings_new_home,
            absolute_cost_savings_compared_to_gas_heating_new_home,
            specific_new_home_heating_demand,
            heating_area,
            new_home_heating_demand,
            new_home_heating_temp,
            electricity_cost,
            gas_cost,
            
        ],
        [
            "Renovations",
            regular_heatpump_renovation_annual_energy,
            regular_heatpump_renovation_annual_co2,
            improved_heatpump_renovation_annual_energy,
            improved_heatpump_renovation_annual_co2,
            relative_energy_savings_renovation,
            relative_co2_savings_renovation,
            relative_cost_savings_renovation,
            relative_cost_savings_compared_to_gas_heating_renovation,
            absolute_energy_savings_renovation,
            absolute_co2_savings_renovation,
            absolute_cost_savings_renovation,
            absolute_cost_savings_compared_to_gas_heating_renovation,
            specific_renovation_heating_demand,
            heating_area,
            renovation_heating_demand,
            renovation_heating_temp,
            
        ],
    ]
    # round all values to 2 decimal places
    for i in range(len(single_home_results)):
        for j in range(1, len(single_home_results[i])):
            single_home_results[i][j] = round(single_home_results[i][j], 2)
            
    
    
    # let's add titles to each row
    rows = ["",
               "regular Heatpump energy consumption [kWh/year]", 
               "regular Heatpump CO2 emissions [kg/year]", 
               "improved Heatpump energy consumption [kWh/year]", 
               "improved Heatpump CO2 emissions [kg/year]", 
               "relative energy savings [%]", 
               "relative CO2 savings [%]", 
               "relative cost savings [%]",
               "relative savings compared to gas heating [%]",
               "absolute energy savings [kWh/year]", 
               "absolute CO2 savings [kg/year]",                                  
               "absolute cost savings [€/year]",
               "absolute savings compared to gas heating [€/year]"]
    
    # let's add a few rows containing the relevant assumptions regarding price, heating demand, heating area, etc.
    rows.extend(["specific heating demand [kWh/year]",
                 "heating area [m²]",
                 "heat demand [kWh/year]",
                 "heating temperature [°C]",
                 "electricity_cost [€/kWh_el]",
                 "gas_cost [€/kWh_th]",
    ])
                 
                    
    
    single_home_results.insert(0,rows)
    # now we create a dataframe and export it to excel
    single_home_results_df = pd.DataFrame(single_home_results)
    # this produces a sheet which has rows and columns swapped so let's transpose it
    single_home_results_df = single_home_results_df.transpose()
    single_home_results_df.to_excel("output/single-home-results.xlsx", index=False)

    results = [
        [
            year,
            new_home_heatpumps[i],
            renovation_heatpumps[i],
            regular_heatpump_new_home_annual_energy * new_home_heatpumps[i] + regular_heatpump_renovation_annual_energy * renovation_heatpumps[i],
            regular_heatpump_new_home_annual_co2 * new_home_heatpumps[i] + regular_heatpump_renovation_annual_co2 * renovation_heatpumps[i],
            improved_heatpump_new_home_annual_energy * new_home_heatpumps[i] + improved_heatpump_renovation_annual_energy * renovation_heatpumps[i],
            improved_heatpump_new_home_annual_co2 * new_home_heatpumps[i] + improved_heatpump_renovation_annual_co2 * renovation_heatpumps[i],
            ((regular_heatpump_new_home_annual_energy - improved_heatpump_new_home_annual_energy) * new_home_heatpumps[i] + (regular_heatpump_renovation_annual_energy - improved_heatpump_renovation_annual_energy) * renovation_heatpumps[i])/1e6,
            ((regular_heatpump_new_home_annual_co2 - improved_heatpump_new_home_annual_co2) * new_home_heatpumps[i] + (regular_heatpump_renovation_annual_co2 - improved_heatpump_renovation_annual_co2) * renovation_heatpumps[i])/1e3,
        ]
        for year, i in enumerate(range(years), start=2026)
    ]

    # we want to calculate the cumulative savings saved that year.
    # since the installed heat pumps continue to provide savings in the coming years, 
    # we need to sum up the savings from the previous years

    results_cumulative=results.copy()
    for i in range(1, len(results)):
        results_cumulative[i][3] = sum(results[j][3] for j in range(i + 1))
        results_cumulative[i][4] = sum(results[j][4] for j in range(i + 1))
        results_cumulative[i][5] = sum(results[j][5] for j in range(i + 1))
        results_cumulative[i][6] = sum(results[j][6] for j in range(i + 1))
        results_cumulative[i][7] = sum(results[j][7] for j in range(i + 1))
        results_cumulative[i][8] = sum(results[j][8] for j in range(i + 1))

    #store results to excel sheet in ouput/results.xlsx
    results_df = pd.DataFrame(
        results_cumulative,
        columns=[
            "Year",
            "New homes",
            "Renovations",
            "Regular energy consumption [kWh]",
            "Regular CO2 emissions [kg]",
            "Improved energy consumption",
            "Improved CO2 emissions",
            "Energy savings [GWh]",
            "CO2 savings [Tons]",
        ],
    )
    
    
    results_df.to_excel("output/cumulative-benefit-forecast.xlsx", index=False)
    
    

In [33]:
main()

regular heat pump rennovation annual energy cost 2942.0721376549636
heat demand in new homes: 5490 kWh
heat demand in renovations: 20130 kWh
cost for gas heating in new homes: 697.23 €
cost for gas heating in renovations: 2556.51 €

relative energy savings in new homes: 12.86%
relative co2 savings in new homes: 12.86%
relative cost savings in new homes: 12.86%
relative cost savings compared to gas heating in new homes: 40.62%
relative energy savings in renovations: 21.74%
relative co2 savings in renovations: 21.75%
relative cost savings in renovations: 21.74%
relative cost savings compared to gas heating in renovations: 9.94%

absolute energy savings in new homes: 152.75 kWh
absolute co2 savings in new homes: 57.64 kg
absolute cost savings in new homes: 61.1 €
absolute cost savings compared to gas heating in new homes: 283.22 €
absolute energy savings in renovations: 1599.18 kWh
absolute co2 savings in renovations: 603.21 kg
absolute cost savings in renovations: 639.67 €
absolute cost 

In [34]:
import numpy as np
# Crankcase heater power rating (W)
crankcase_heater_power = 70

# Assumption: Crankcase heater operates when the ambient temperature is below 10°C
heater_operating_hours_per_day = [6 if temp < 10 else 0 for temp in monthly_ambient_temps]

monthly_hdd = [max(base_temperature - temp, 0) for temp in monthly_ambient_temps]

# Total number of days the crankcase heater operates in a year
operating_days_per_month = np.array(heater_operating_hours_per_day) * np.array(list(monthly_hdd))
total_operating_days = sum(operating_days_per_month)

# Energy wasted on crankcase heaters in a year (kWh)
energy_wasted = crankcase_heater_power * total_operating_days / 1000